<a href="https://colab.research.google.com/github/yongsa-nut/TU_CN408_GenAI_671/blob/main/HW3_Prompt_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prompt Engineering (10 Points)

## Setting up

In [ ]:
#setting up
# Uncomment below if you are doing API key
# !pip install anthropic
# !pip install python-dotenv
!pip install -U google-cloud-aiplatform "anthropic[vertex]" # Don't forget to restart session

## **Skip this part if you are not using Anthopic's API**


## Safely storing your API key

For best practice to keep sensitive information, like API keys, separate from your codebase. One common approach is to store the API key in a `.env` file and load it using the `python-dotenv` package. Here's how you can set it up:

Create a new file called `.env`.

Add your API key to the newly created .env file using the following format:

`ANTHROPIC_API_KEY=put-your-api-key-here`

Make sure to save the `.env` file

Finally upload `.env.` file to your google colab. It should look like this.

![key_trial.png](https://drive.google.com/uc?export=view&id=1qZywP0bsjyaVAuAOu9EE-RvjwSL0VUOY)

In [ ]:
from dotenv import load_dotenv
import os
from google.colab import data_table
import pandas as pd
import re
import time

data_table.enable_dataframe_formatter() # Allow you to view dataset on colab nicely

In [ ]:
# Load your API key
load_dotenv()
my_api_key = os.getenv("ANTHROPIC_API_KEY")

In [ ]:
from anthropic import Anthropic

client = Anthropic(
    api_key=my_api_key
)

# Helper function to get response from Claude. DO NOT CHANGE.
# You will be using Claude 3 Haiku for this homework
# Temperature is set to 0 so that the answer will be the same.
def get_response(user_prompt, system_prompt="", prefill="",temperature=0):
  response = client.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=1500,
    temperature=temperature,
    system= system_prompt,
    messages=[
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": prefill} #Only need this if we plan to do prefill
    ]
  )
  return response.content[0].text

# Testing if your api key is working
print(get_response("Hello"))

Hello! How can I assist you today?


## Google Vertex Setup

In [ ]:
!gcloud auth application-default login

In [ ]:
!gcloud auth application-default set-quota-project xxxxxxxxx # replace the last one with your project ID

In [3]:
from anthropic import AnthropicVertex

project_id = "xxxxxxxxx" # replace this with your project ID
region = "us-east5"  # Two region for Haiku ["us-east5", "europe-west1"]

client = AnthropicVertex(project_id=project_id, region=region)

# Helper function to get response from Claude. DO NOT CHANGE.
# You will be using Claude 3 Haiku for this homework
# Temperature is set to 0 so that the answer will be the same.
def get_response(user_prompt, system_prompt="", prefill="",temperature=0):
  response = client.messages.create(
    model="claude-3-haiku@20240307",
    max_tokens=1500,
    temperature=temperature,
    system= system_prompt,
    messages=[
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": prefill} #Only need this if we plan to do prefill
    ]
  )
  return response.content[0].text

# Testing if your api key is working
print(get_response("Hello"))

Hello! How can I assist you today?


# Part 1: Prompting Techniques (8 points)

- Questions are from [Claude Exercises](https://docs.google.com/spreadsheets/d/19jzLgRruG9kjUQNKtCg1ZjdD6l6weA6qRXG5zLIAhC8/edit?usp=sharing)
- 0.5 point per question

### **Q1**. Basic Prompt Structure

**Q1.1** Get Claude to count to three

In [ ]:
prompt = "" #Fill in your prompt here
print(get_response(prompt))

**Q1.2** Modify the system prompt to make **Claude respond like it's a 3 year old child**.

In [ ]:
system_prompt = "" #Fill in your prompt here
prompt = "How big is the sky?"
print(get_response(prompt, system_prompt))

The sky does not have a definitive size, as it refers to the expanse of space above the Earth's surface. Here are some key points about the size of the sky:

- The sky extends infinitely in all directions. It has no physical boundaries or edges.

- The visible sky that we can see from Earth's surface is limited by the horizon. This visible portion of the sky is often referred to as the celestial sphere.

- The celestial sphere appears to have a radius of about 4,000-6,000 miles (6,400-9,600 km) from the perspective of an observer on Earth. This is the distance to the horizon.

- However, the actual size of the universe, which the sky represents, is essentially infinite. Astronomers estimate the observable universe has a diameter of about 93 billion light-years.

- The sky contains the Earth's atmosphere, which extends about 6,200 miles (10,000 km) above the surface. Beyond that is the vacuum of space.

So in summary, while the visible sky has a finite size from an earthly perspective, 

### **Q2**. Being Clear and Direct

**Q2.1** Adapt the system prompt to make Claude output its answer in Thai.


In [ ]:
system_prompt = "" #Fill in your prompt here
prompt = "Hello Claude, how are you?"
print(get_response(prompt, system_prompt))

Hello! As an AI language model, I don't have feelings or a physical form, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


**Q2.2** Modify the basketball player prompt so that Claude doesn't equivocate at all and responds with **ONLY the name of one specific player**, with no other words or punctuation.


In [ ]:
prompt = "Who is the best basketball player of all time?"
print(get_response(prompt))

There is no universally agreed upon "best basketball player of all time." It's a highly debated topic, as there have been many all-time great players throughout the history of the sport. Some of the players often considered among the greatest of all time include:

- Michael Jordan - Widely regarded as one of the greatest, if not the greatest, player ever. He won 6 NBA championships with the Chicago Bulls.

- LeBron James - One of the most dominant and well-rounded players of the modern era. He has won 4 NBA titles with 3 different teams.

- Kareem Abdul-Jabbar - Legendary center who won 6 NBA championships and holds the record for most career points scored.

- Wilt Chamberlain - Dominant big man who holds numerous NBA records including most points in a game (100).

- Magic Johnson - Pioneering point guard who led the "Showtime" Lakers to 5 NBA titles.

- Bill Russell - Defensive anchor and 11-time NBA champion with the Boston Celtics.

There are good arguments for several players as th

**Q2.3** Modify the prompt so that Claude responds with as long a response as you can muster. Claude's response should be over 800 words.


In [ ]:
prompt = "Can you write me a story?"
response = get_response(prompt)
print(f"Number of words = {len(response.split())}")
print(response)

### **Q3** Role Prompting

**Q3.1** In some instances, Claude may struggle with mathematics, even simple mathematics. Below, Claude incorrectly assesses the math problem as correctly solved, even though there's an obvious arithmetic mistake in the second step. Note that Claude actually catches the mistake when going through step-by-step, but doesn't jump to the conclusion that the overall solution is wrong.

Adapt the text in system prompt box to make Claude grade the solution as **incorrectly solved**, rather than **correctly solved**.

In [ ]:
system_prompt = "" #Fill in your prompt here
prompt = """Is this equation solved correctly below?
2x - 3 = 9
2x = 6
x = 3"""
print(get_response(prompt, system_prompt))

Yes, the equation is solved correctly.

Let's go through the steps:

Given equation:
2x - 3 = 9

Step 1: Isolate the variable term by adding 3 to both sides.
2x - 3 + 3 = 9 + 3
2x = 12

Step 2: Divide both sides by 2 to find the value of x.
2x / 2 = 12 / 2
x = 6

Therefore, the solution to the equation 2x - 3 = 9 is x = 3.


### **Q4**. Separating Data from Instructions

**Q4.1** Write a prompt that will take in a variable called "{{TOPIC}}" and output a haiku about the topic. This exercise is just meant to test your understanding of the variable templating structure.

In [ ]:
TOPIC = "pig"
prompt = f""  # use f-string to create a prompt with variables
print(get_response(prompt))

**Q4.2** Fix the prompt below by adding XML tags so that Claude produces the right answer.

Try not to change anything else about the prompt. The messy and mistake-ridden writing is intentional, so you can see how Claude reacts to such mistakes.

In [ ]:
QUESTION = "ar cn brown?"
prompt = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"
print(get_response(prompt))

I'm afraid I don't fully understand your question. Could you please rephrase it in a clearer way? I'd be happy to try to answer your question about dogs, but I need a more coherent and grammatically correct question to work with. Please take your time and provide the details you want me to address. I'll do my best to give you a short, direct answer.


**Q4.3** Fix the prompt **WITHOUT adding XML tags**. Instead, remove only one or two words from the prompt.

In [ ]:
QUESTION = "ar cn brown?"
prompt = f"Hia its me i have a q about dogs jkaerjv {QUESTION} jklmvca tx it help me muhch much atx fst fst answer short short tx"
print(get_response(prompt))

I'm afraid I don't fully understand your question. Could you please rephrase it in a clearer way? I'd be happy to try to answer your question about dogs, but I need a more coherent and grammatically correct question to work with. Please take your time and provide the details you want me to address. I'll do my best to give you a short, direct answer.


### **Q5**: Formatting Output & Speaking for Claude

**Q5.1** Forced to make a choice, Claude designates Michael Jordan as the best basketball player of all time. Can we get Claude to pick someone else?

Modify the "best basketball player" prompt and use the prefill technique (putting words after "Assistant:") to compell Claude to make a detailed argument that the best basketball player of all time is Stephen Curry.

In [6]:
prompt = "Who is the best basketball player of all time? Please choose one specific player."
prefill = ""
print(get_response(prompt, prefill=prefill))

In my opinion, the best basketball player of all time is Michael Jordan. Here's a brief overview of why I believe this:

- Michael Jordan is widely regarded as one of the most dominant and influential players in NBA history. He is a 6-time NBA champion, 5-time MVP, 14-time All-Star, and 10-time scoring champion.

- Jordan's combination of athleticism, skill, competitiveness, and clutch performances made him nearly unstoppable during his prime years in the 1980s and 1990s. He is considered one of the greatest scorers the game has ever seen.

- Jordan's impact on the sport of basketball and popular culture is unparalleled. He helped drive the NBA's global popularity and is considered a cultural icon. His "Air Jordan" brand with Nike is one of the most successful athlete-endorsed products ever.

- While players like LeBron James, Kareem Abdul-Jabbar, and others have strong cases, Jordan's sustained excellence, individual accolades, and cultural impact set him apart in the minds of many ba

**Q5.2** Modify the haiku prompt below and **use XML tags so that Claude writes two haikus about the animal instead of just one**. It should be clear where one poem ends and the other begins.


In [ ]:
ANIMAL = 'cats'
prompt = f"Please write a haiku about {ANIMAL}. Put it in <haiku> tags." #Modify this one
prefill = "<haiku>"
print(get_response(prompt, prefill=prefill))

**Q5.3** Modify the haiku prompt in the highlighted template box below so that Claude produces two haikus about two different animals.

Use {ANIMAL1} as a stand-in for the first substitution, and {ANIMAL2} as a stand-in for the second substitution.

In [ ]:
ANIMAL1 = 'cat'
ANIMAL2 = 'dog'
prompt = f"Please write a haiku about {ANIMAL1}. Put it in <haiku> tags." #Modify this one
print(get_response(prompt))

**Q5.4**: In this exercise, we'll be instructing Claude to sort emails into the following categories:

* (A) Pre-sale question
* (B) Broken or defective item
* (C) Billing question
* (D) Other (please explain)

For the first part of the exercise, change the prompt below to **make Claude output the correct classification and ONLY the classification**. Your answer needs to include the letter (A - D) of the correct choice, with the parentheses, as well as the name of the category.


In [ ]:
prompt = f"Please classify this email as either green or blue: {{EMAIL}}" #Modify this!
prefill = ""
# Hint:
## 1) How will Claude know what categories you want to use?
## 2) Be sure to tell to only include the classification
## 3) Consider using prefill to force Claude to only response with the classification immedietely

email1 = "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement."
email2 = "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?"
email3 = "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???"
email4 = "How did I get here I am not good with computer.  Halp."

print(get_response(prompt.format(EMAIL=email1)), prefill=prefill) # The answer is B
print(get_response(prompt.format(EMAIL=email2)), prefill=prefill) # The answer is D or A
print(get_response(prompt.format(EMAIL=email3)), prefill=prefill) # The answer is C
print(get_response(prompt.format(EMAIL=email4)), prefill=prefill) # The answer is D

### **Q6** Using examples

**Q6.1**:  In this exercise, we're going to refine the output of the above prompt to yield an answer formatted exactly how we want it. This time, we're going to use "few-shot" examples of emails + proper classification (and formatting) to get Claude to output the correct

Use XML to make Claude wrap just the letter of the correct classification in `<answer></answer>` tags. Refer to the above exercise if you forget which letter is correct for each email. For instance, the answer to the first email should contain the exact string `<answer>B</answer>`.

In [ ]:
prompt = f"Please classify this email as either green or blue: {{EMAIL}}" # Start from your previous answer.
prefill = "" # If you prefill, Claude won't actually output that as part of its response.
# Hint: you could ask Claude to help come with examples if you struggle to come up with one.

email1 = "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement."
email2 = "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?"
email3 = "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???"
email4 = "How did I get here I am not good with computer.  Halp."

print(get_response(prompt.format(EMAIL=email1)), prefill=prefill) # The answer is B
print(get_response(prompt.format(EMAIL=email2)), prefill=prefill) # The answer is D or A
print(get_response(prompt.format(EMAIL=email3)), prefill=prefill) # The answer is C
print(get_response(prompt.format(EMAIL=email4)), prefill=prefill) # The answer is D

## **Q7**: Dealing with Hallucinations

**Q7.1** Adapt the prompt below to fix Claude's hallucination issue by giving Claude an out. (Renaissance is Beyoncé's seventh studio album, not her eighth.)

Hint: an out = allowing Claude to say I don't know.


In [ ]:
prompt = "In what year did star performer Beyoncé release her eighth studio album?"
print(get_response(prompt))

Beyoncé released her eighth studio album, Renaissance, in 2022.

The album was released on July 29, 2022 through Parkwood Entertainment and Columbia Records.

Some key details:

- Renaissance is Beyoncé's eighth solo studio album, following her previous album Lemonade which was released in 2016.

- The album was announced by Beyoncé in June 2022, with the release date of July 29, 2022.

- Renaissance has been praised by critics as a return to Beyoncé's dance-oriented roots, drawing influences from disco, house, and other electronic dance music genres.

So in summary, Beyoncé's eighth studio album Renaissance was released in the year 2022.


**Q7.2** Modify the prompt below to fix Claude's hallucination issue by asking for citations. The correct answer is that subscribers went up 49-fold.


In [ ]:
# Hint:
# 1) Ask it to read or use the document
# 2) Then ask Claude to list the relevant quote from the document
# 3) Then consider whether the quote answers the question.
# 4) Finally, ask it to write an answer
# 5) use XML tags as you see fit. (e.g., <thinking>, <scatchpad>, <answer>)

prompt = """User: From December 2018 to December 2022, by what amount did Matterport's subscribers grow?

<document>
Matterport SEC filing 10-K 2023
Item 1. Business
Our Company
Matterport is leading the digitization and datafication of the built world. We believe the digital transformation of the built world will fundamentally change the way people interact with buildings and the physical spaces around them.
Since its founding in 2011, Matterport’s pioneering technology has set the standard for digitizing, accessing and managing buildings, spaces and places online. Our platform’s innovative software, spatial data-driven data science, and 3D capture technology have broken down the barriers that have kept the largest asset class in the world, buildings and physical spaces, offline and underutilized for many years. We believe the digitization and datafication of the built world will continue to unlock significant operational efficiencies and property values, and that Matterport is the platform to lead this enormous global transformation.
The world is rapidly moving from offline to online. Digital transformation has made a powerful and lasting impact across every business and industry today. According to International Data Corporation, or IDC, over $6.8 trillion of direct investments will be made on digital transformation from 2020 to 2023, the global digital transformation spending is forecasted to reach $3.4 trillion in 2026 with a five-year compound annual growth rate (“CAGR”) of 16.3%, and digital twin investments are expected to have a five-year CAGR of 35.2%. With this secular shift, there is also growing demand for the built world to transition from physical to digital. Nevertheless, the vast majority of buildings and spaces remain offline and undigitized. The global building stock, estimated by Savills to be $327 trillion in total property value as of 2021, remains largely offline today, and we estimate that less than 0.1% is penetrated by digital transformation.
Matterport was among the first to recognize the increasing need for digitization of the built world and the power of spatial data, the unique details underlying buildings and spaces, in facilitating the understanding of buildings and spaces. In the past, technology advanced physical road maps to the data-rich, digital maps and location services we all rely on today. Matterport now digitizes buildings, creating a data-rich environment to vastly increase our understanding and the full potential of each and every space we capture. Just as we can instantly, at the touch of a button, learn the fastest route from one city to another or locate the nearest coffee shops, Matterport’s spatial data for buildings unlocks a rich set of insights and learnings about properties and spaces worldwide. In addition, just as the geo-spatial mapping platforms of today have opened their mapping data to industry to create new business models such as ridesharing, e-commerce, food delivery marketplaces, and even short-term rental and home sharing, open access to Matterport’s structured spatial data is enabling new opportunities and business models for hospitality, facilities management, insurance, construction, real estate and retail, among others.
We believe the total addressable market opportunity for digitizing the built world is over $240 billion, and could be as high as $1 trillion as the market matures at scale. This is based on our analysis, modeling and understanding of the global building stock of over 4 billion properties and 20 billion spaces in the world today. With the help of artificial intelligence (“AI”), machine learning (“ML”) and deep learning (“DL”) technologies, we believe that, with the additional monetization opportunities from powerful spatial data-driven property insights and analytics, the total addressable market for the digitization and datafication of the built world will reach more than $1 trillion.

Our spatial data platform and capture of digital twins deliver value across a diverse set of industries and use cases. Large retailers can manage thousands of store locations remotely, real estate agencies can provide virtual open houses for hundreds of properties and thousands of visitors at the same time, property developers can monitor the entirety of the construction process with greater detail and speed, and insurance companies can more precisely document and evaluate claims and underwriting assessments with efficiency and precision. Matterport delivers the critical digital experience, tools and information that matter to our subscribers about properties of virtually any size, shape, and location worldwide.
For nearly a decade, we have been growing our spatial data platform and expanding our capabilities in order to create the most detailed, accurate, and data-rich digital twins available. Moreover, our 3D reconstruction process is fully automated, allowing our solution to scale with equal precision to millions of buildings and spaces of any type, shape, and size in the world. The universal applicability of our service provides Matterport significant scale and reach across diverse verticals and any geography. As of December 31, 2022, our subscriber base had grown approximately 39% to over 701,000 subscribers from 503,000 subscribers as of December 31, 2021, with our digital twins reaching more than 170 countries. We have digitized more than 28 billion square feet of space across multiple industries, representing significant scale and growth over the rest of the market.

As we continue to transform buildings into data worldwide, we are extending our spatial data platform to further transform property planning, development, management and intelligence for our subscribers across industries to become the de facto building and business intelligence engine for the built world. We believe the demand for spatial data and resulting insights for enterprises, businesses and institutions across industries, including real estate, architecture, engineering and construction (“AEC”), retail, insurance and government, will continue to grow rapidly.
We believe digitization and datafication represent a tremendous greenfield opportunity for growth across this massive category and asset class. From the early stages of design and development to marketing, operations, insurance and building repair and maintenance, our platform’s software and technology provide subscribers critical tools and insights to drive cost savings, increase revenues and optimally manage their buildings and spaces. We believe that hundreds of billions of dollars in unrealized utilization and operating efficiencies in the built world can be unlocked through the power of our spatial data platform. Our platform and data solutions have universal applicability across industries and building categories, giving Matterport a significant advantage as we can address the entirety of this large market opportunity and increase the value of what we believe to be the largest asset class in the world.
With a demonstrated track record of delivering value to our subscribers, our offerings include software subscription, data licensing, services and product hardware. As of December 31, 2022, our subscriber base included over 24% of Fortune 1000 companies, with less than 10% of our total revenue generated from our top 10 subscribers. We expect more than 80% of our revenue to come from our software subscription and data license solutions by 2025. Our innovative 3D capture products, the Pro2 and Pro3 Cameras, have played an integral part in shaping the 3D building and property visualization ecosystem. The Pro2 and Pro3 Cameras have driven adoption of our solutions and have generated the unique high-quality and scaled data set that has enabled Cortex, our proprietary AI software engine, to become the pioneering engine for digital twin creation. With this data advantage initially spurred by the Pro2 Camera, we have developed a capture device agnostic platform that scales and can generate new building and property insights for our subscribers across industries and geographies.
We have recently experienced rapid growth. Our subscribers have grown approximately 49-fold from December 31, 2018 to December 31, 2022. Our revenue increased by approximately 22% to $136.1 million for the year ended December 31, 2022, from approximately $111.2 million for the year ended December 31, 2021. Our gross profit decreased by $8.1 million or 14%, to $51.8 million for the year ended December 31, 2022, from $60.0 million for the year ended December 31, 2021, primarily attributable to certain disruptive and incremental costs due to the global supply chain constraints in fiscal year 2022. Our ability to retain and grow the subscription revenue generated by our existing subscribers is an important measure of the health of our business and our future growth prospects. We track our performance in this area by measuring our net dollar expansion rate from the same set of customers across comparable periods. Our net dollar expansion rate of 103% for the three months ended December 31, 2022 demonstrates the stickiness and growth potential of our platform.
Our Industry and Market Opportunity
Today, the vast majority of buildings and spaces remain undigitized. We estimate our current serviceable addressable market includes approximately 1.3 billion spaces worldwide, primarily from the real estate and travel and hospitality sectors. With approximately 9.2 million spaces under management as of December 31, 2022, we are continuing to penetrate the global building stock and expand our footprint across various end markets, including residential and commercial real estate, facilities management, retail, AEC, insurance and repair, and travel and hospitality. We estimate our total addressable market to be more than 4 billion buildings and 20 billion spaces globally, yielding a more than $240 billion market opportunity. We believe that as Matterport’s unique spatial data library and property data services continue to grow, this opportunity could increase to more than $1 trillion based on the size of the building stock and the untapped value creation available to buildings worldwide. The constraints created by the COVID-19 pandemic have only reinforced and accelerated the importance of our scaled 3D capture solution that we have developed for diverse industries and markets over the past decade.

Our Spatial Data Platform
Overview
Our technology platform uses spatial data collected from a wide variety of digital capture devices to transform physical buildings and spaces into dimensionally accurate, photorealistic digital twins that provide our subscribers access to previously unavailable building information and insights.
As a first mover in this massive market for nearly a decade, we have developed and scaled our industry-leading 3D reconstruction technology powered by Cortex, our proprietary AI-driven software engine that uses machine learning to recreate a photorealistic, 3D virtual representation of an entire building structure, including contents, equipment and furnishings. The finished product is a detailed and dynamic replication of the physical space that can be explored, analyzed and customized from a web browser on any device, including smartphones. The power to manage even large-scale commercial buildings is in the palm of each subscriber’s hands, made possible by our advanced technology and breakthrough innovations across our entire spatial data technology stack.
Key elements of our spatial data platform include:
•Bringing offline buildings online. Traditionally, our customers needed to conduct in-person site visits to understand and assess their buildings and spaces. While photographs and floor plans can be helpful, these forms of two-dimensional (“2D”) representation have limited information and tend to be static and rigid, and thus lack the interactive element critical to a holistic understanding of each building and space. With the AI-powered capabilities of Cortex, our proprietary AI software, representation of physical objects is no longer confined to static 2D images and physical visits can be eliminated. Cortex helps to move the buildings and spaces from offline to online and makes them accessible to our customers in real-time and on demand from anywhere. After subscribers scan their buildings, our visualization algorithms accurately infer spatial positions and depths from flat, 2D imagery captured through the scans and transform them into high- fidelity and precise digital twin models. This creates a fully automated image processing pipeline to ensure that each digital twin is of professional grade image quality.
•Driven by spatial data. We are a data-driven company. Each incremental capture of a space grows the richness and depth of our spatial data library. Spatial data represents the unique and idiosyncratic details that underlie and compose the buildings and spaces in the human- made environment. Cortex uses the breadth of the billions of data points we have accumulated over the years to improve the 3D accuracy of our digital twins. We help our subscribers pinpoint the height, location and other characteristics of objects in their digital twin. Our sophisticated algorithms also deliver significant commercial value to our subscribers by generating data-based insights that allow them to confidently make assessments and decisions about their properties. For instance, property developers can assess the amount of natural heat and daylight coming from specific windows, retailers can ensure each store layout is up to the same level of code and brand requirements, and factories can insure machinery layouts meet specifications and location guidelines. With approximately 9.2 million spaces under management as of December 31, 2022, our spatial data library is the clearinghouse for information about the built world.
•Powered by AI and ML. Artificial intelligence and machine learning technologies effectively utilize spatial data to create a robust virtual experience that is dynamic, realistic, interactive, informative and permits multiple viewing angles. AI and ML also make costly cameras unnecessary for everyday scans—subscribers can now scan their spaces by simply tapping a button on their smartphones. As a result, Matterport is a device agnostic platform, helping us more rapidly scale and drive towards our mission of digitizing and indexing the built world.
Our value proposition to subscribers is designed to serve the entirety of the digital building lifecycle, from design and build to maintenance and operations, promotion, sale, lease, insure, repair, restore, secure and finance. As a result, we believe we are uniquely positioned to grow our revenue with our subscribers as we help them to discover opportunities to drive short- and long-term return on investment by taking their buildings and spaces from offline to online across their portfolios of properties.
Ubiquitous Capture
Matterport has become the standard for 3D space capture. Our technology platform empowers subscribers worldwide to quickly, easily and accurately digitize, customize and manage interactive and dimensionally accurate digital twins of their buildings and spaces.
The Matterport platform is designed to work with a wide range of LiDAR, spherical, 3D and 360 cameras, as well as smartphones, to suit the capture needs of all of our subscribers. This provides the flexibility to capture a space of any size, scale, and complexity, at anytime and anywhere.
•Matterport Pro3 is our newest 3D camera that scans properties faster than earlier versions to help accelerate project completion. Pro3 provides the highest accuracy scans of both indoor and outdoor spaces and is designed for speed, fidelity, versatility and accuracy. Capturing 3D data up to 100 meters away at less than 20 seconds per sweep, Pro3’s ultra-fast, high-precision LiDAR sensor can run for hours and takes millions of measurements in any conditions.
•Matterport Pro2 is our proprietary 3D camera that has been used to capture millions of spaces around the world with a high degree of fidelity, precision, speed and simplicity. Capable of capturing buildings more than 500,000 square feet in size, it has become the camera of choice for many residential, commercial, industrial and large-scale properties.
•360 Cameras. Matterport supports a selection of 360 cameras available in the market. These affordable, pocket sized devices deliver precision captures with high fidelity and are appropriate for capturing smaller homes, condos, short-term rentals, apartments, and more. The spherical lens image capture technology of these devices gives Cortex robust, detailed image data to transform panoramas into our industry-leading digital twins.
•LEICA BLK360. Through our partnership with Leica, our 3D reconstruction technology and our AI powered software engine, Cortex, transform this powerful LiDAR camera into an ultra-precise capture device for creating Matterport digital twins. It is the solution of choice for AEC professionals when exacting precision is required.
•Smartphone Capture. Our capture apps are commercially available for both iOS and Android. Matterport’s smartphone capture solution has democratized 3D capture, making it easy and accessible for anyone to digitize buildings and spaces with a recent iPhone device since the initial introduction of Matterport for iPhone in May 2020. In April 2021, we announced the official release of the Android Capture app, giving Android users the ability to quickly and easily capture buildings and spaces in immersive 3D. In February 2022, we launched Matterport Axis, a motorized mount that holds a smartphone and can be used with the Matterport Capture app to capture 3D digital twins of any physical space with increased speed, precision, and consistency.
Cortex and 3D Reconstruction (the Matterport Digital Twin)
With a spatial data library, as of December 31, 2022, of approximately 9.2 million spaces under management, representing approximately 28 billion captured square feet of space, we use our advanced ML and DL technologies to algorithmically transform the spatial data we capture into an accurate 3D digital reproduction of any physical space. This intelligent, automated 3D reconstruction is made possible by Cortex, our AI-powered software engine that includes a deep learning neural network that uses our spatial data library to understand how a building or space is divided into floors and rooms, where the doorways and openings are located, and what types of rooms are present, such that those forms are compiled and aligned with dimensional accuracy into a dynamic, photorealistic digital twin. Other components of Cortex include AI-powered computer vision technologies to identify and classify the contents inside a building or space, and object recognition technologies to identify and segment everything from furnishings and equipment to doors, windows, light fixtures, fire suppression sprinklers and fire escapes. Our highly scalable artificial intelligence platform enables our subscribers to tap into powerful, enhanced building data and insights at the click of a button.

The Science Behind the Matterport Digital Twin: Cortex AI Highlights
Matterport Runs on Cortex
Cortex is our AI-powered software engine that includes a precision deep learning neural network to create digital twins of any building or space. Developed using our proprietary spatial data captured with our Pro2 and Pro3 cameras, Cortex delivers a high degree of precision and accuracy while enabling 3D capture using everyday devices.
Generic neural networks struggle with 3D reconstruction of the real world. Matterport-optimized networks deliver more accurate and robust results. More than just raw training data, Matterport’s datasets allow us to develop new neural network architectures and evaluate them against user behavior and real-world data in millions of situations.
•Deep learning: Connecting and optimizing the detailed neural network data architecture of each space is key to creating robust, highly accurate 3D digital twins. Cortex evaluates and optimizes each 3D model against Matterport’s rich spatial data aggregated from millions of buildings and spaces and the human annotations of those data provided by tens of thousands of subscribers worldwide. Cortex’s evaluative abilities and its data-driven optimization of 3D reconstruction yield consistent, high-precision results across a wide array of building configurations, spaces and environments.
•Dynamic 3D reconstruction: Creating precise 3D spatial data at scale from 2D visuals and static images requires a combination of photorealistic, detailed data from multiple viewpoints and millions of spaces that train and optimize Cortex’s neural network and learning capabilities for improved 3D reconstruction of any space. Cortex’s capabilities combined with real-time spatial alignment algorithms in our 3D capture technology create an intuitive “preview” of any work in progress, allowing subscribers to work with their content interactively and in real-time.
•Computer vision: Cortex enables a suite of powerful features to enhance the value of digital twins. These include automatic measurements for rooms or objects in a room, automatic 2D-from-3D high-definition photo gallery creation, auto face blurring for privacy protection, custom videos, walkthroughs, auto room labeling and object recognition.
•Advanced image processing: Matterport’s computational photography algorithms create a fully automated image processing pipeline to help ensure that each digital twin is of professional grade image quality. Our patented technology makes 3D capture as simple as pressing a single button. Matterport’s software and technology manage the remaining steps, including white balance and camera-specific color correction, high dynamic range tone mapping, de-noising, haze removal, sharpening, saturation and other adjustments to improve image quality.
Spatial Data and AI-Powered Insights
Every Matterport digital twin contains extensive information about a building, room or physical space. The data uses our AI-powered Cortex engine. In addition to the Matterport digital twin itself, our spatial data consists of precision building geometry and structural detail, building contents, fixtures and condition, along with high-definition imagery and photorealistic detail from many vantage points in a space. Cortex employs a technique we call deep spatial indexing. Deep spatial indexing uses artificial intelligence, computer vision and deep learning to identify and convey important details about each space, its structure and its contents with precision and fidelity. We have created a robust spatial data standard that enables Matterport subscribers to harness an interoperable digital system of record for any building.
In addition to creating a highly interactive digital experience for subscribers through the construction of digital twins, we ask ourselves two questions for every subscriber: (1) what is important about their building or physical space and (2) what learnings and insights can we deliver for this space? Our AI-powered Cortex engine helps us answer these questions using our spatial data library to provide aggregated property trends and operational and valuation insights. Moreover, as the Matterport platform ecosystem continues to expand, our subscribers, partners and other third-party developers can bring their own tools to further the breadth and depth of insights they can harvest from our rich spatial data layer.
Extensible Platform Ecosystem
Matterport offers the largest and most accurate library of spatial data in the world, with, as of December 31, 2022, approximately 9.2 million spaces under management and approximately 28 billion captured square feet. The versatility of our spatial data platform and extensive enterprise software development kit and application programming interfaces (“APIs”) has allowed us to develop a robust global ecosystem of channels and partners that extend the Matterport value proposition by geography and vertical market. We intend to continue to deploy a broad set of workflow integrations with our partners and their subscribers to promote an integrated Matterport solution across our target markets. We are also developing a third-party software marketplace to extend the power of our spatial data platform with easy-to-deploy and easy-to-access Matterport software add-ons. The marketplace enables developers to build new applications and spatial data mining tools, enhance the Matterport 3D experience, and create new productivity and property management tools that supplement our core offerings. These value-added capabilities created by third-party developers enable a scalable new revenue stream, with Matterport sharing the subscription and services revenue from each add-on that is deployed to subscribers through the online marketplace. The network effects of our platform ecosystem contributes to the growth of our business, and we believe that it will continue to bolster future growth by enhancing subscriber stickiness and user engagement.
Examples of Matterport add-ons and extensions include:
•Add-ons: Encircle (easy-to-use field documentation tools for faster claims processing); WP Matterport Shortcode (free Wordpress plugin that allows Matterport to be embedded quickly and easily with a Matterport shortcode), WP3D Models (WordPress + Matterport integration plugin); Rela (all-in-one marketing solution for listings); CAPTUR3D (all-in-one Content Management System that extends value to Matterport digital twins); Private Model Emded (feature that allows enterprises to privately share digital twins with a large group of employees on the corporate network without requiring additional user licenses); Views (new workgroup collaboration framework to enable groups and large organizations to create separate, permissions-based workflows to manage different tasks with different teams); and Guided Tours and Tags (tool to elevate the visitor experience by creating directed virtual tours of any commercial or residential space tailored to the interests of their visitors). We unveiled our private beta integration with Amazon Web Services (AWS) IoT TwinMaker to enable enterprise customers to seamlessly connect IoT data into visually immersive and dimensionally accurate Matterport digital twin.
•Services: Matterport ADA Compliant Digital Twin (solution to provide American Disability Act compliant digital twins) and Enterprise Cloud Software Platform (reimagined cloud software platform for the enterprise that creates, publishes, and manages digital twins of buildings and spaces of any size of shape, indoors or outdoors).
Our Competitive Strengths
We believe that we have a number of competitive strengths that will enable our market leadership to grow. Our competitive strengths include:
•Breadth and depth of the Matterport platform. Our core strength is our all-in-one spatial data platform with broad reach across diverse verticals and geographies such as capture to processing to industries without customization. With the ability to integrate seamlessly with various enterprise systems, our platform delivers value across the property lifecycle for diverse end markets, including real estate, AEC, travel and hospitality, repair and insurance, and industrial and facilities. As of December 31, 2022, our global reach extended to subscribers in more than 170 countries, including over 24% of Fortune 1000 companies.
•Market leadership and first-mover advantage. Matterport defined the category of digitizing and datafying the built world almost a decade ago, and we have become the global leader in the category. As of December 31, 2022, we had over 701,000 subscribers on our platform and approximately 9.2 million spaces under management. Our leadership is primarily driven by the fact that we were the first mover in digital twin creation. As a result of our first mover advantage, we have amassed a deep and rich library of spatial data that continues to compound and enhance our leadership position.
•Significant network effect. With each new capture and piece of data added to our platform, the richness of our dataset and the depth of insights from our spaces under management grow. In addition, the combination of our ability to turn data into insights with incremental data from new data captures by our subscribers enables Matterport to develop features for subscribers to our platform. We were a first mover in building a spatial data library for the built world, and our leadership in gathering and deriving insights from data continues to compound and the relevance of those insights attracts more new subscribers.
•Massive spatial data library as the raw material for valuable property insights. The scale of our spatial data library is a significant advantage in deriving insights for our subscribers. Our spatial data library serves as vital ground truth for Cortex, enabling Matterport to create powerful 3D digital twins using a wide range of camera technology, including low-cost digital and smartphone cameras. As of December 31, 2022, our data came from approximately 9.2 million spaces under management and approximately 28 billion captured square feet. As a result, we have taken property insights and analytics to new levels, benefiting subscribers across various industries. For example, facilities managers significantly reduce the time needed to create building layouts, leading to a significant decrease in the cost of site surveying and as-built modeling. AEC subscribers use the analytics of each as-built space to streamline documentation and collaborate with ease.
•Global reach and scale. We are focused on continuing to expand our AI-powered spatial data platform worldwide. We have a significant presence in North America, Europe and Asia, with leadership teams and a go-to-market infrastructure in each of these regions. We have offices in London, Singapore and several across the United States, and we are accelerating our international expansion. As of December 31, 2022, we had over 701,000 subscribers in more than 170 countries. We believe that the geography-agnostic nature of our spatial data platform is a significant advantage as we continue to grow internationally.
•Broad patent portfolio supporting 10 years of R&D and innovation. As of December 31, 2022, we had 54 issued and 37 pending patent applications. Our success is based on almost 10 years of focus on innovation. Innovation has been at the center of Matterport, and we will continue to prioritize our investments in R&D to further our market leading position.
•Superior capture technology. Matterport’s capture technology platform is a software framework that enables support for a wide variety of capture devices required to create a Matterport digital twin of a building or space.
This includes support for LiDAR cameras, 360 cameras, smartphones, Matterport Axis and the Matterport Pro2 and Pro3 cameras. The Pro2 camera was foundational to our spatial data advantage, and we have expanded that advantage with an array of Matterport-enabled third-party capture devices. In August 2022, we launched and began shipment of our Pro3 Camera along with major updates to our industry-leading digital twin cloud platform. The Matterport Pro3 Camera is an advanced 3D capture device, which includes faster boot time, swappable batteries, and a lighter design. The Pro3 camera can perform both indoors and outdoors and is designed for speed, fidelity, versatility and accuracy. Along with our Pro2 Camera, we expect that future sales of our Pro3 Camera will continue to drive increased adoption of our solutions. Matterport is democratizing the 3D capture experience, making high-fidelity and high-accuracy 3D digital twins readily available for any building type and any subscriber need in the property life cycle. While there are other 3D capture solution providers, very few can produce true, dimensionally accurate 3D results, and fewer still can automatically create a final product in photorealistic 3D, and at global scale. This expansive capture technology offering would not be possible without our rich spatial data library available to train the AI-powered Cortex engine to automatically generate accurate digital twins from photos captured with a smartphone or 360 camera.
</document>
"""

print(get_response(prompt))

## Part 2: Classify Customer Intent (2 points)

For this problem, your task is to create a prompt for Claude 3 Haiku to classify the customer query into category and intent. The dataset is from https://huggingface.co/datasets/bitext/Bitext-customer-support-llm-chatbot-training-dataset.
The code below will filter the data out to only 3 categories and 9 intents as follows.

* ORDER category
  * cancel_order
  * change_order
  * place_order
  * track_order
* PAYMENT category
  * check_payment_methods
  * payment_issue
* REFUND category
  * check_refund_policy
  * get_refund
  * track_refund

For each intent, there are 5 data points (total 45). This is due to the rate limit of free tier account.

**Input-Ouput Requirement:**

- Input will be pass to `{{QUERY}}` so your prompt must include it.
- Output for category must be inside `<category>` tags and for output for intent must be inside `<intent>` tags
- See the code below for more details
- Due to rate limit, the code below will send the query every 0.2 seconds. You may not want to wait until the end if your prompt doesn't work correctly. You can stop and change your prompt.

**Evaluation:**
- For this problem, we will check two things: the accuracy in predicting category and the accuracy in predicting intent. The baseline for predicting category is 33.34% and for intent is 11.11%. **To get points, you must perform better than the baselines.** If you do everything correctly, you will like get close to 100%.
- The code for evaluation is provided for you.

### Load the data

In [ ]:
# You will get a warning but you can simply ignore it.
df = pd.read_csv("hf://datasets/bitext/Bitext-customer-support-llm-chatbot-training-dataset/Bitext_Sample_Customer_Support_Training_Dataset_27K_responses-v11.csv")

### Preprocess Data

In [ ]:
# drop the column 'response' and 'flags'. We won't use it for this question.
df = df.drop(columns=['response', 'flags'])

#Filter the dataset to only ORDER, PAYMENT, REFUND
df = df[df['category'].isin(['ORDER', 'PAYMENT', 'REFUND'])]

m = 5 # 5 each intent
def sample_m_from_each_class(group, m):
    if len(group) >= m:
        return group.sample(m, random_state=42)  # DO NOT CHANGE random_state (seed)
    else:
        return group  # If the group has less than m samples, return all of them

# Group by 'intent' and apply the sampling function
sampled_df = df.groupby('intent').apply(sample_m_from_each_class, m=m).reset_index(drop=True)

In [ ]:
# View the dataset
sampled_df

,instruction,category,intent
0,I cannot afford purchase {{Order Number}},ORDER,cancel_order
1,"I have bought some item, I have to cancel orde...",ORDER,cancel_order
2,"I do not want this item, cancel order {{Order ...",ORDER,cancel_order
3,I need assistance with canceling purchase {{Or...,ORDER,cancel_order
4,were can i cancel purchase {{Order Number}},ORDER,cancel_order
5,I have to delete several items from purchase {...,ORDER,change_order
6,updating purchase {{Order Number}},ORDER,change_order
7,update purchase00123842,ORDER,change_order
8,I want assistance editing purchase {{Order Nu...,ORDER,change_order
9,I have to delete an item from order {{Order Nu...,ORDER,change_order


In [ ]:
# There are 3 categories and 9 intents
pd.crosstab(sampled_df['intent'], sampled_df['category'])

category,ORDER,PAYMENT,REFUND
intent,,,
cancel_order,5,0,0
change_order,5,0,0
check_payment_methods,0,5,0
check_refund_policy,0,0,5
get_refund,0,0,5
payment_issue,0,5,0
place_order,5,0,0
track_order,5,0,0
track_refund,0,0,5


### **Your prompt here**
- Feel free to do whatever you want

In [ ]:
# Fill in your prompts here

prompt = """
"""
system_prompt = ""
prefill = ""

### **Prediction and Evaluation**
- Once you fill out your prompt simply run the code below

In [ ]:
# if you want to test your prompt quickly use this block
index = 0 # change this to test different data (0 - 44)
fill_prompt = prompt.replace('{{QUERY}}', df['instruction'][index])
response = get_response(fill_prompt, system_prompt, prefill)
print(response)

<answer>
<category>ORDER</category>
<intent>cancel_order</intent>
</answer>


In [ ]:
pause = 13 # Pause time to not exceed rate limit (5 per minute)

# First we randomly shuffle the data
shuffled_df = sampled_df.sample(frac=1, random_state=77).reset_index(drop=True)
shuffled_df['predicted_category'] = ''
shuffled_df['predicted_intent'] = ''

# Then we generate predictions

def predict(prompt, system_prompt, prefill, df, echo=True):
  for i in range(len(df)):
    fill_prompt = prompt.replace('{{QUERY}}', df['instruction'][i])
    response = get_response(fill_prompt, system_prompt, prefill)

    # Extract XML tags
    content = re.search('<intent>(.*?)</intent>', response).group(1)
    df['predicted_intent'][i] = content
    content = re.search('<category>(.*?)</category>', response).group(1)
    df['predicted_category'][i] = content

    if echo:
      print("Query:", df['instruction'][i])
      print(response)

    time.sleep(pause) # waiting

# Then we generate predictions
predict(prompt, system_prompt, prefill, shuffled_df, echo = True)

In [ ]:
# Check the prediction
shuffled_df

In [ ]:
# Evaluation
def evaluation(df):
  acc_category =sum(df['category'] == df['predicted_category']) / len(df)
  acc_intent = sum(df['intent'] == df['predicted_intent']) / len(df)
  return acc_category, acc_intent

acc_category, acc_intent = evaluation(shuffled_df)
print(f"Accuracy in predicting category: {acc_category:.2f}")
print(f"Accuracy in predicting intent: {acc_intent:.2f}")